<a href="https://colab.research.google.com/github/ML-Student1/Steven-Schaerer/blob/Machine-Learning/FCCClassificationFlowerProject.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#DS separates flowers into 3 different classes of species (Setosa, Versicolor, Virginica) Flower info: Sepal Length, Sepal Width, Petal Length, Petal Width
#Given flower info, which flower class is it most likely to be?

In [23]:
%tensorflow_version 2.x #Only required if you're in a notebook

`%tensorflow_version` only switches the major version: 1.x or 2.x.
You set: `2.x #Only required if you're in a notebook`. This will be interpreted as: `2.x`.


TensorFlow is already loaded. Please restart the runtime to change versions.


In [24]:
import tensorflow as tf

import pandas as pd

In [25]:
CSV_COLUMN_NAMES = ['SepalLength', 'SepalWidth', 'PetalLength', 'PetalWidth', 'Species']
SPECIES = ['Setosa', 'Versicolor', 'Virginica'] #Define constants

In [26]:
train_path = tf.keras.utils.get_file(
    "iris_training.csv", "https://storage.googleapis.com/download.tensorflow.org/data/iris_training.csv")
test_path = tf.keras.utils.get_file(
    "iris_test.csv", "https://storage.googleapis.com/download.tensorflow.org/data/iris_test.csv")
train = pd.read_csv(train_path, names=CSV_COLUMN_NAMES, header=0)
test = pd.read_csv(test_path, names=CSV_COLUMN_NAMES, header=0) #Keras (TF module) used to grab datasets and read them into PD

In [27]:
train.head()

,SepalLength,SepalWidth,PetalLength,PetalWidth,Species
0,6.4,2.8,5.6,2.2,2
1,5.0,2.3,3.3,1.0,1
2,4.9,2.5,4.5,1.7,2
3,4.9,3.1,1.5,0.1,0
4,5.7,3.8,1.7,0.3,0


In [28]:
train_y = train.pop('Species')
test_y = test.pop('Species')
train.head() #'pop' the species column off to use as label

,SepalLength,SepalWidth,PetalLength,PetalWidth
0,6.4,2.8,5.6,2.2
1,5.0,2.3,3.3,1.0
2,4.9,2.5,4.5,1.7
3,4.9,3.1,1.5,0.1
4,5.7,3.8,1.7,0.3


In [29]:
train.shape #120 entries with 4 features

(120, 4)

In [30]:
train_y.head()

0    2
1    1
2    2
3    0
4    0
Name: Species, dtype: int64

In [31]:
#Input Function

def input_fn(features, labels, training=True, batch_size=256): #convert the inputs to a dataset
    dataset = tf.data.Dataset.from_tensor_slices((dict(features), labels))

#Shuffle and repeat if you are in training mode
    if training:
      dataset = dataset.shuffle(1000).repeat()
    return dataset.batch(batch_size)

In [32]:
#Feature Columns - describe how to use the input

my_feature_columns = []
for key in train.keys():
  my_feature_columns.append(tf.feature_column.numeric_column(key=key))
print(my_feature_columns)

[NumericColumn(key='SepalLength', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None), NumericColumn(key='SepalWidth', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None), NumericColumn(key='PetalLength', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None), NumericColumn(key='PetalWidth', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None)]


In [33]:
#Building the Model - Classification Model
#DNNClassifeier (Deep Neural Network...may not be linear correspondence in data)
#LinearClassifier (Classification rather than regression, labels, probability)
#Build a DNN with 2 hidden layers with 30 and 10 hidden nodes each

classifier = tf.estimator.DNNClassifier(
    feature_columns=my_feature_columns,
    #Two hidden layers of 30 and 10 nodes respectively (Hidden Layers build architecture of Neural Network)...Input Layer, Hidden Layer, Output Layer
    hidden_units=[30, 10],
    n_classes=3) #3 classes for the flowers

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': '/tmp/tmp2_oumgcq', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_experimental_max_worker_delay_secs': None, '_session_creation_timeout_secs': 7200, '_checkpoint_save_graph_def': True, '_service': None, '_cluster_spec': ClusterSpec({}), '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


In [34]:
#Training the Model
#lambda anonymous function defined in one line (after colon is what function does, one line function here)

classifier.train(
    input_fn=lambda: input_fn(train, train_y, training=True),
    steps=5000)

INFO:tensorflow:Calling model_fn.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Calling checkpoint listeners before saving checkpoint 0...
INFO:tensorflow:Saving checkpoints for 0 into /tmp/tmp2_oumgcq/model.ckpt.
INFO:tensorflow:Calling checkpoint listeners after saving checkpoint 0...
INFO:tensorflow:loss = 1.46164, step = 0
INFO:tensorflow:global_step/sec: 422.723
INFO:tensorflow:loss = 1.0821763, step = 100 (0.242 sec)
INFO:tensorflow:global_step/sec: 478.484
INFO:tensorflow:loss = 1.039747, step = 200 (0.209 sec)
INFO:tensorflow:global_step/sec: 503.556
INFO:tensorflow:loss = 0.9923842, step = 300 (0.196 sec)
INFO:tensorflow:global_step/sec: 441.714
INFO:tensorflow:loss = 0.9768605, step = 400 (

In [37]:
eval_result = classifier.evaluate(input_fn=lambda: input_fn(test, test_y, training=False))

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Starting evaluation at 2021-10-27T16:54:07
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from /tmp/tmp2_oumgcq/model.ckpt-5000
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Inference Time : 0.22198s
INFO:tensorflow:Finished evaluation at 2021-10-27-16:54:07
INFO:tensorflow:Saving dict for global step 5000: accuracy = 0.8, average_loss = 0.5546199, global_step = 5000, loss = 0.5546199
INFO:tensorflow:Saving 'checkpoint_path' summary for global step 5000: /tmp/tmp2_oumgcq/model.ckpt-5000


In [38]:
print('\nTest set accuracy: {accuracy:0.3f} \n'.format(**eval_result))


Test set accuracy: 0.800 



In [44]:
#Predictions
#Convert inputs to a dataset without labels

def input_fn(features, batch_size=256):
    return tf.data.Dataset.from_tensor_slices(dict(features)).batch(batch_size) #create dataset from those features, dict, followed by batch size (no y value/labels, won't know the label, model provides answer.)

features = ['SepalLength', 'SepalWidth', 'PetalLength', 'PetalWidth']
predict = {}

print("Please type numeric values as prompted.")
for feature in features:
  valid = True
  while valid:  #for each feature, wait to get some valid response
    val = input(feature + ": ") 
    if not val.isdigit(): valid = False

  predict[feature] = [float(val)] #add to dictionary once we get valid response, predicting whatever feature is equal to a list (whatever value was)  Predict method works on multiple things and needs to be placed in the list

predictions = classifier.predict(input_fn=lambda: input_fn(predict))
for pred_dict in predictions:  #every prediction comes back as a dictionary, 
  class_id = pred_dict['class_ids'][0] #class id is equal to class ID dictionary at 0
  probability = pred_dict['probabilities'][class_id] #probability equal to pred dict 'probs' of class id

  print('Prediction is "{}" ({:.1f}%))'.format(
      SPECIES[class_id], 100 * probability) #100 * for integer value
  )

Please type numeric values as prompted.
SepalLength: 2.4
SepalWidth: 2.6
PetalLength: 6.5
PetalWidth: 6.3
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from /tmp/tmp2_oumgcq/model.ckpt-5000
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
Prediction is "Virginica" (97.3%))
